In [2]:
import pandas as pd

# Load the dataset
df = pd.read_csv(r"C:\Users\X1 CARBON TOUCH\Desktop\Introduction to Data Science with Jupyter Notebook\spacex.csv")

# Clean column names
df.columns = df.columns.str.strip()

# Display column names to debug
print("Columns in dataset:", df.columns.tolist())

# Question 3: How many launches went to geosynchronous orbit?
if 'Orbit' in df.columns:
    geo_launches = df[df['Orbit'].str.contains('GEO', case=False, na=False)]
    print("Number of launches to geosynchronous orbit:", len(geo_launches))
else:
    print("🚨 Column 'Orbit' not found in the dataset!")

# Question 4: How many missions landed on a drone ship?
landing_col = [col for col in df.columns if 'Landing' in col]
if landing_col:
    drone_landing_success = df[df[landing_col[0]].str.contains("Success (drone ship)", na=False)]
    print("Number of successful drone ship landings:", len(drone_landing_success))
else:
    print("🚨 Landing outcome column not found in the dataset!")

Columns in dataset: ['Flight Number,Launch Date,Launch Time,Launch Site,Vehicle Type,Payload Name,Payload Type,Payload Mass (kg),Payload Orbit,Customer Name,Customer Type,Customer Country,Mission Outcome,Failure Reason,Landing Type,Landing Outcome']
🚨 Column 'Orbit' not found in the dataset!
Number of successful drone ship landings: 0


C:\Users\X1 CARBON TOUCH\AppData\Local\Temp\ipykernel_8820\817507148.py:22: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  drone_landing_success = df[df[landing_col[0]].str.contains("Success (drone ship)", na=False)]


In [4]:
import requests
import pandas as pd
from datetime import datetime

# Get past launches and payloads
launches_url = "https://api.spacexdata.com/v4/launches/past"
payloads_url = "https://api.spacexdata.com/v4/payloads"

# API calls
launches = requests.get(launches_url).json()
payloads = requests.get(payloads_url).json()

# Filter launches up to 2020 only
filtered_launches = [
    launch for launch in launches
    if datetime.strptime(launch['date_utc'], '%Y-%m-%dT%H:%M:%S.%fZ').year <= 2020
]

# --------- Question 3: GTO Orbit Launches ---------
# Collect payload IDs from filtered launches
payload_ids = []
for launch in filtered_launches:
    payload_ids.extend(launch['payloads'])

# Convert payloads to DataFrame
payload_df = pd.json_normalize(payloads)

# Filter to only payloads used in filtered launches
filtered_payload_df = payload_df[payload_df['id'].isin(payload_ids)]

# Count how many are GTO orbit
gto_count = filtered_payload_df[filtered_payload_df['orbit'] == 'GTO']['id'].nunique()

# --------- Question 4: Successful ASDS Landings ---------
successful_drone_landings = 0
for launch in filtered_launches:
    for core in launch['cores']:
        if core['landing_type'] == 'ASDS' and core['landing_success'] == True:
            successful_drone_landings += 1

# Output results
print("Launches to GTO orbit (up to 2020):", gto_count)
print("Successful ASDS landings (up to 2020):", successful_drone_landings)

Launches to GTO orbit (up to 2020): 36
Successful ASDS landings (up to 2020): 48
